# Getting Started with Machine Learning Zoo

This notebook demonstrates how to use the Machine Learning Zoo for an end-to-end ML task.

## 1. Setup and Imports

In [ ]:
import torch
from src.models.registry import get_model
from src.training.trainer import TrainingOrchestrator
print("Setup complete.")

## 2. Load a Model
We can load any model from the Zoo using the unified registry.

In [ ]:
model_name = "LSTM"
config = {
    "feature_dim": 10,
    "hidden_dim": 32,
    "output_dim": 1,
    "num_layers": 2
}
model_cls = get_model(model_name)
model = model_cls(config)
print(f"Loaded {model_name} model architecture.")

## 3. Training with Orchestrator
The `TrainingOrchestrator` handles DP, DDP, and DeepSpeed automatically.

In [ ]:
orchestrator = TrainingOrchestrator(model, epochs=5)
print("Orchestrator ready for training.")